In [1]:
import pyomo.environ as pyo
import numpy as np
from scipy.sparse import coo_matrix
from pyomo.contrib.pynumero.interfaces.external_grey_box import ExternalGreyBoxModel
from pyomo.contrib.pynumero.interfaces.external_grey_box import ExternalGreyBoxBlock

## Grey-box model


In [22]:
class LogDetModel(ExternalGreyBoxModel):
    def __init__(self, use_exact_derivatives=True,verbose=False):
        self._use_exact_derivatives = use_exact_derivatives
        self.verbose = verbose

        # For use with exact Hessian
        self._output_con_mult_values = np.zeros(1)

        if not use_exact_derivatives:
            raise NotImplementedError("use_exact_derivatives == False not supported")
        
    def input_names(self):
        """List of names, same order as in set_input_values
        """
        return ['a', 'b', 'c']

    def equality_constraint_names(self):
        """String names corresponding to any residuals for this external model.
        """
        return [ ]
    
    def output_names(self):
        return ['log_det']

    def set_output_constraint_multipliers(self, output_con_multiplier_values):
        assert len(output_con_multiplier_values) == 1
        np.copyto(self._output_con_mult_values, output_con_multiplier_values)

    def finalize_block_construction(self, pyomo_block):
        # set lower bounds on the variables
        pyomo_block.inputs['a'].setlb(10)
        pyomo_block.inputs['b'].setlb(1)
        pyomo_block.inputs['c'].setlb(20)

        # set upper bounds on the variables
        pyomo_block.inputs['a'].setub(10)
        pyomo_block.inputs['b'].setub(1)
        pyomo_block.inputs['c'].setub(20)

        # initialize the variables
        #pyomo_block.inputs['a'].value = 10
        #pyomo_block.inputs['b'].value = -10
        #pyomo_block.inputs['c'].value = 10
        
        #pyomo_block.add_const1 = pyo.Constraint(rule=pyomo_block.inputs['a']==5)
        #pyomo_block.add_const2 = pyo.Constraint(rule=pyomo_block.inputs['b']==1)
        #pyomo_block.add_const3 = pyo.Constraint(rule=pyomo_block.inputs['c']==10)

    def set_input_values(self, input_values):
        self._input_values = list(input_values)
        
    def evaluate_equality_constraints(self):
        """Compute the residuals from the moodel using values set in input_values, return as a np array
        """
        return None
    
    def evaluate_outputs(self):
        """Compute outputs from the model using values set in input_values, return as a numpy array
        """
        a = self._input_values[0]
        b = self._input_values[1]
        c = self._input_values[2]

        # form matrix
        M = np.array([[a,b],[b,c]])

        # compute log determinant
        (sign, logdet) = np.linalg.slogdet(M)

        if self.verbose:
            print("\n Consider M =\n",M)
            print("   logdet = ",logdet,"\n")

        return np.asarray([logdet], dtype=np.float64)

    def evaluate_jacobian_equality_constraints(self):
        return None

    def evaluate_jacobian_outputs(self):
        """compute the derivatives of the ooutputs with respect to thee inputs 
        Return a scipy matrix with the rows in the order of the output variables,
        cols in the order of the input variables
        """

        a = self._input_values[0]
        b = self._input_values[1]
        c = self._input_values[2]

        if self._use_exact_derivatives:

            M = np.array([[a,b],[b,c]])

            # compute inverse
            Minv = np.linalg.pinv(M)

            row = np.zeros(3)
            col = np.zeros(3)
            data = np.zeros(3)
            row[0], col[0], data[0] = (0, 0, Minv[0,0])
            row[1], col[1], data[1] = (0, 1, 2*Minv[0,1])
            row[2], col[2], data[2] = (0, 2, Minv[1,1])
            return coo_matrix((data, (row, col)), shape=(1,3))

    '''
    def evaluate_hessian_outputs(self):

        a = self._input_values[0]
        b = self._input_values[1]
        c = self._input_values[2]

        if self._use_exact_derivatives:

            # form matrix
            M = np.array([[a,b],[b,c]])

            # compute inverse
            Minv = np.linalg.pinv(M)

            # Let f(X) = logdet(X)
            # Minv = [ [df/da, df/db],
            #          [df/db, df/da] ]

            # Want Hessian
            # [ [d2f/da2,  d2d/dadb],
            #   [d2f/dadb, d2f/db2 ] ]

            # Let S1 = dX / da, thus
            # S1 = [ [1, 0], 
            #        [0, 1] ]
            #
            # Minv @ S1 @ Minv =
            # [ [df2/da2, df2/dbda], 
            #   [df2/dbda, df2/da2]

            # Similarly
            # Let S2 = dX / db, thus
            # S2 = [ [0, 1], 
            #        [1, 0] ]
            #
            # Minv @ S2 @ Minv =
            # [ [df2/dadb, df2/db2], 
            #   [df2/db2, df2/dadb]

            S1 = np.array([[1, 0],[0,0]])
            temp1 = -Minv @ S1 @ Minv

            S2 = np.array([[0,1],[1,0]])
            temp2 = -Minv @ S2 @ Minv

            S3 = np.array([[0, 0],[0,1]])
            temp3 = -Minv @ S3 @ Minv

            # Extract values
            df2da2 = temp1[0,0]
            df2dadb = 2*temp1[0,1]
            df2db2 = 2*temp1[1,1]
            df2dc2 = temp3[1,1]
            df2dbdc = 2*temp3[0,1]
            df2dadc = temp2[0,1] - df2db2

            lam = self._output_con_mult_values
        
            # Note: Only return low triangular of Hessian
            nnz = 6
            irow = np.zeros(nnz, dtype=np.int64)
            jcol = np.zeros(nnz, dtype=np.int64)
            data = np.zeros(nnz, dtype=np.float64)

            idx = 0
            irow[idx], jcol[idx], data[idx] = (0, 0, lam[0]*df2da2)
            idx += 1
            irow[idx], jcol[idx], data[idx] = (1, 0, lam[0]*df2dadb)
            idx += 1
            irow[idx], jcol[idx], data[idx] = (1, 1, lam[0]*df2db2)
            idx += 1
            irow[idx], jcol[idx], data[idx] = (2, 0, lam[0]*df2dadc)
            idx += 1
            irow[idx], jcol[idx], data[idx] = (2, 1, lam[0]*df2dbdc)
            idx += 1
            irow[idx], jcol[idx], data[idx] = (2, 2, lam[0]*df2dc2)
            idx += 1

        assert idx == nnz
        hess = coo_matrix( (data, (irow,jcol)), shape=(3,3) )
        return hess
    '''

In [23]:
def maximize_log_det(show_solver_log=True, additional_options={}):
    m = pyo.ConcreteModel()

    # create a block to store the external grey box model
    m.my_block = ExternalGreyBoxBlock(external_model=LogDetModel())
    
    #m.my_block.set_input_values([10,1,20])
    
    # add objective to maximize log det
    m.obj = pyo.Objective(expr=m.my_block.outputs['log_det'], sense=pyo.maximize)

    solver = pyo.SolverFactory('cyipopt')

    solver.config.options['hessian_approximation'] = 'limited-memory'

    for k,v in additional_options.items():
        solver.config.options[k] = v
    
    solver.tee=True
        
    results = solver.solve(m, tee=True)
    pyo.assert_optimal_termination(results)
    return m, solver

In [24]:
m, solver1 = maximize_log_det(additional_options={'max_iter':5000})




In [25]:
print("Grey-box log det:", m.my_block.outputs['log_det'].value)

a = m.my_block.inputs['a'].value
b = m.my_block.inputs['b'].value
c = m.my_block.inputs['c'].value

test = np.asarray([[a,b],[b,c]])
print(test)

det = np.linalg.det(test)
print('Real log det:', np.log(det))

Grey-box log det: 5.293304824724492
[[10.  1.]
 [ 1. 20.]]
Real log det: 5.293304824724492


## Check

In [ ]:
# form matrix
def jaco(a, b, c):
    M = np.array([[a,b],[b,c]])

    # compute inverse
    Minv = np.linalg.pinv(M)

    row = np.zeros(3)
    col = np.zeros(3)
    data = np.zeros(3)
    row[0], col[0], data[0] = (0, 0, Minv[0,0])
    row[1], col[1], data[1] = (0, 1, Minv[0,1])
    row[2], col[2], data[2] = (0, 2, Minv[1,1])
    return coo_matrix((data, (row, col)), shape=(1,3))

print(jaco(20,10,50))

In [ ]:
from scipy.sparse import csr_matrix
def hes(a,b,c):
    # form matrix
    M = np.array([[a,b],[b,c]])

    # compute inverse
    Minv = np.linalg.pinv(M)

    S1 = np.array([[1, 0],[0,0]])
    temp1 = -Minv @ S1 @ Minv

    S2 = np.array([[0,1],[1,0]])
    temp2 = -Minv @ S2 @ Minv
    
    S3 = np.array([[0, 0],[0,1]])
    temp3 = -Minv @ S3 @ Minv

    # Extract values
    df2da2 = temp1[0,0]
    df2dadb = temp1[0,1]
    df2db2 = temp1[1,1]
    df2dc2 = temp3[1,1]
    df2dbdc = temp3[0,1]
    df2dadc = temp2[0,1] - df2db2


    # Note: Only return low triangular of Hessian
    nnz = 6
    irow = np.zeros(nnz, dtype=np.int64)
    jcol = np.zeros(nnz, dtype=np.int64)
    data = np.zeros(nnz, dtype=np.float64)

    idx = 0
    irow[idx], jcol[idx], data[idx] = (0, 0, df2da2)
    idx += 1
    irow[idx], jcol[idx], data[idx] = (1, 0, df2dadb)
    idx += 1
    irow[idx], jcol[idx], data[idx] = (1, 1, df2db2)
    idx += 1
    irow[idx], jcol[idx], data[idx] = (2, 0, df2dadc)
    idx += 1
    irow[idx], jcol[idx], data[idx] = (2, 1, df2dbdc)
    idx += 1
    irow[idx], jcol[idx], data[idx] = (2, 2, df2dc2)
    idx += 1

    assert idx == nnz
    hess = coo_matrix( (data, (irow,jcol)), shape=(3,3) )
    
    return csr_matrix(hess).todense()



In [ ]:
test = hes(20, 10, 50)

test[0,1] = test[1,0]
test[0,2] = test[2,0]
test[1,2] = test[2,1]

print(test)

## Verify

In [ ]:
def finite_diff(a, b, c, step=0.001):
    
    a_up = a*(1+step)
    a_lo = a*(1-step)
    
    det_up_a = np.log(np.linalg.det(np.asarray([[a_up,b],[b,c]])))
    det_lo_a = np.log(np.linalg.det(np.asarray([[a_lo,b],[b,c]])))
    
    a_dev = (det_up_a-det_lo_a)/(2*step*a)
    
    b_up = b*(1+step)
    b_lo = b*(1-step)
    
    det_up_b = np.log(np.linalg.det(np.asarray([[a,b_up],[b_up,c]])))
    det_lo_b = np.log(np.linalg.det(np.asarray([[a,b_lo],[b_lo,c]])))
    
    b_dev = (det_up_b-det_lo_b)/(2*step*b)
    
    c_up = c*(1+step)
    c_lo = c*(1-step)
    
    det_up_c = np.log(np.linalg.det(np.asarray([[a,b],[b,c_up]])))
    det_lo_c = np.log(np.linalg.det(np.asarray([[a,b],[b,c_lo]])))
    
    c_dev = (det_up_c-det_lo_c)/(2*step*c)
    
    return a_dev, b_dev, c_dev

In [ ]:
test1 = np.asarray([[20,10*1.001], [10*1.001, 50]])
test2 =  np.asarray([[20,10*0.999], [10*0.999, 50]])

a = np.log(np.linalg.det(test1))-np.log(np.linalg.det(test2))
print(a/(2*0.001*10))

In [ ]:
der_a, der_b, der_c = finite_diff(20, 10, 50, step=0.01)
print(np.asarray([[der_a, der_b],[der_b, der_c]]))

In [ ]:
def finite_hes(a, b, c, step=0.001):
    
    a_up = a*(1+step)
    a_lo = a*(1-step)
    
    b_up = b*(1+step)
    b_lo = b*(1-step)
    
    c_up = c*(1+step)
    c_lo = c*(1-step)
    
    # d(f)/da*da
    perta_a_up, _, _ = finite_diff(a_up, b, c)
    perta_a_lo, _, _ = finite_diff(a_lo, b, c)
    
    f_aa = (perta_a_up-perta_a_lo)/(2*step*a)
    
    # da*db
    pertb_a_up, _, _ = finite_diff(a, b_up, c)
    pertb_a_lo, _, _ = finite_diff(a, b_lo, c)
    
    f_ab = (pertb_a_up-pertb_a_lo)/(2*step*b)
    print(f_ab)
    
    # da*dc
    pertc_a_up, _, _ = finite_diff(a, b, c_up)
    pertc_a_lo, _, _ = finite_diff(a, b, c_lo)
    
    f_ac = (pertc_a_up-pertc_a_lo)/(2*step*c)
    
    # da*db, it should be the same as f_ab; I recompute to verify they are the same. 
    _, perta_b_up, _ = finite_diff(a_up, b, c)
    _, perta_b_lo, _ = finite_diff(a_lo, b, c)
    
    f_ba = (perta_b_up-perta_b_lo)/(2*step*a)
    
    print(f_ba)
    
    # db*db
    _, pertb_b_up, _ = finite_diff(a, b_up, c)
    _, pertb_b_lo, _ = finite_diff(a, b_lo, c)
    
    f_bb = (pertb_b_up-pertb_b_lo)/(2*step*b)
    
    # db*dc
    _, pertc_b_up, _ = finite_diff(a, b, c_up)
    _, pertc_b_lo, _ = finite_diff(a, b, c_lo)
    
    f_bc = (pertc_b_up-pertc_b_lo)/(2*step*c)
    
    # da*dc, for veification 
    _, _, perta_c_up = finite_diff(a_up, b, c)
    _, _, perta_c_lo = finite_diff(a_lo, b, c)
    
    f_ca = (perta_c_up-perta_c_lo)/(2*step*a)
    
    # db*dc, for verification 
    
    _, _, pertb_c_up = finite_diff(a, b_up, c)
    _, _, pertb_c_lo = finite_diff(a, b_lo, c)
    
    f_cb = (pertb_c_up-pertb_c_lo)/(2*step*b)
    
    # dc*dc
    
    _, _, pertc_c_up = finite_diff(a, b, c_up)
    _, _, pertc_c_lo = finite_diff(a, b, c_lo)
    
    f_cc = (pertc_c_up-pertc_c_lo)/(2*step*c)
    
    hes = [[f_aa, f_ab, f_ac], [f_ba, f_bb, f_bc], [f_ca, f_cb, f_cc]]
    
    return hes

In [ ]:
hessian = finite_hes(20,10,50)

print(np.asarray(hessian))

In [ ]:
test1 = np.asarray([[10, 0], [0, 10]])

print(np.linalg.det(test1))

test2 = np.asarray([[10,1], [1,10]])
print(np.linalg.det(test2))

In [ ]:
mat = np.asarray([[1, 0], [0, -1]])

print((np.linalg.det(mat)))

In [ ]:
print((np.linalg.slogdet(mat)))